In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import multiprocessing
import parmap
import numpy as np

In [2]:
biomart_ensembl = pd.read_csv('/home/weber/PycharmProjects/ExoCarto/data/0_external/BIOMART_HGNC_ENSEMBL.txt.gz', sep='\t', compression='gzip')
biomart_ensembl = biomart_ensembl.dropna(subset=['HGNC ID'])
biomart_ensembl['HGNC ID'] = biomart_ensembl['HGNC ID'].astype(str)
biomart_ensembl['HGNC ID'] = biomart_ensembl['HGNC ID'].str.replace('HGNC:', '')
biomart_ensembl

,Gene stable ID,HGNC ID,HGNC symbol
0,ENSG00000210049,7481,MT-TF
1,ENSG00000211459,7470,MT-RNR1
2,ENSG00000210077,7500,MT-TV
3,ENSG00000210082,7471,MT-RNR2
4,ENSG00000209082,7490,MT-TL1
...,...,...,...
67150,ENSG00000288200,49825,SNRPCP10
67152,ENSG00000288476,35781,RPS3AP15
67155,ENSG00000284997,14531,TUBAP2
67156,ENSG00000284805,47786,RNU6-823P


In [ ]:
ptex_file = "/gstock/biolo_datasets/variation/gnomAD/latest/gtex/gtex_gnomad.parquet"
rawdf_ptex = pd.read_parquet(ptex_file)
rawdf_ptex = pd.merge(rawdf_ptex, biomart_ensembl, right_on='HGNC symbol', left_on='symbol')
rawdf_ptex = rawdf_ptex[['HGNC ID', 'symbol', 'ensg', 'chrom', 'pos'] + list(rawdf_ptex.columns)[4:-3]]
rawdf_ptex['HGNC ID'] = rawdf_ptex['HGNC ID'].astype(int)
rawdf_ptex

In [ ]:
original_pext = pd.read_csv('/gstock/biolo_datasets/variation/gnomAD/latest/gtex/all.baselevel.021620.tsv.bgz', compression='gzip', sep='\t', low_memory=False)
pext_modified = original_pext.dropna(subset=[list(original_pext.columns)[-1]])
pext_modified[['chrom', 'pos']] = pext_modified['locus'].str.split(':', expand=True)
pext_modified = pd.merge(pext_modified, biomart_ensembl, right_on='HGNC symbol', left_on='symbol', how='left')
pext_modified = pext_modified[['HGNC ID', 'symbol', 'ensg', 'chrom', 'pos'] + list(pext_modified.columns)[4:-3]]
# original_pext['HGNC ID'] = original_pext['HGNC ID'].astype(int)
pext_modified
pext_modified.loc[:,~pext_modified.columns.duplicated()].to_parquet('/gstock/biolo_datasets/variation/gnomAD/latest/gtex/gtex_gnomad_corrected.parquet')

In [3]:
pext_modified = pd.read_parquet('/gstock/biolo_datasets/variation/gnomAD/latest/gtex/gtex_gnomad_corrected.parquet')
pext_modified

,HGNC ID,symbol,ensg,chrom,pos,Prostate,Spleen,Brain_FrontalCortex_BA9_,SmallIntestine_TerminalIleum,MinorSalivaryGland,...,Brain_Anteriorcingulatecortex_BA24_,Brain_Putamen_basalganglia_,Brain_Caudate_basalganglia_,Colon_Sigmoid,Cervix_Endocervix,Ovary,Esophagus_GastroesophagealJunction,Testis,Brain_Amygdala,mean_proportion
0,None,AL627309.1,ENSG00000237683,1,138530,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.00000,1.0,1.00000
1,None,AL627309.1,ENSG00000237683,1,138531,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.00000,1.0,1.00000
2,None,AL627309.1,ENSG00000237683,1,138532,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.00000,1.0,1.00000
3,None,AL627309.1,ENSG00000237683,1,138533,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.00000,1.0,1.00000
4,None,AL627309.1,ENSG00000237683,1,138534,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.00000,1.0,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38837009,1809,CDY1,ENSG00000172288,Y,27770670,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26316,NaN,0.13158
38837010,1809,CDY1,ENSG00000172288,Y,27770671,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26316,NaN,0.13158
38837011,1809,CDY1,ENSG00000172288,Y,27770672,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26316,NaN,0.13158
38837012,1809,CDY1,ENSG00000172288,Y,27770673,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26316,NaN,0.13158


In [25]:
pext_modified.columns

Index(['HGNC ID', 'symbol', 'ensg', 'chrom', 'pos', 'Prostate', 'Spleen',
       'Brain_FrontalCortex_BA9_', 'SmallIntestine_TerminalIleum',
       'MinorSalivaryGland', 'Artery_Coronary', 'Skin_SunExposed_Lowerleg_',
       'Cells_EBV_transformedlymphocytes', 'Brain_Hippocampus',
       'Esophagus_Muscularis', 'Brain_Nucleusaccumbens_basalganglia_',
       'Artery_Tibial', 'Brain_Hypothalamus', 'Adipose_Visceral_Omentum_',
       'Cervix_Ectocervix', 'Brain_Spinalcord_cervicalc_1_',
       'Brain_CerebellarHemisphere', 'Nerve_Tibial', 'Breast_MammaryTissue',
       'Liver', 'Skin_NotSunExposed_Suprapubic_', 'AdrenalGland', 'Vagina',
       'Pancreas', 'Lung', 'FallopianTube', 'Pituitary', 'Muscle_Skeletal',
       'Colon_Transverse', 'Artery_Aorta', 'Heart_AtrialAppendage',
       'Adipose_Subcutaneous', 'Esophagus_Mucosa', 'Heart_LeftVentricle',
       'Brain_Cerebellum', 'Brain_Cortex', 'Thyroid', 'Brain_Substantianigra',
       'Kidney_Cortex', 'Uterus', 'Stomach', 'WholeBlood', 'B

In [22]:
pext_modified = pext_modified.dropna(subset=['HGNC ID'])
pext_modified['HGNC ID'] = pext_modified['HGNC ID'].astype(int)
pext_modified['pos'] = pext_modified['pos'].astype(int)


In [104]:
pext_modified

,HGNC ID,symbol,ensg,chrom,pos,Prostate,Spleen,Brain_FrontalCortex_BA9_,SmallIntestine_TerminalIleum,MinorSalivaryGland,...,Brain_Anteriorcingulatecortex_BA24_,Brain_Putamen_basalganglia_,Brain_Caudate_basalganglia_,Colon_Sigmoid,Cervix_Endocervix,Ovary,Esophagus_GastroesophagealJunction,Testis,Brain_Amygdala,mean_proportion
780,31275,OR4F29,ENSG00000235249,1,367659,1.0,1.0,1.0,1.0,NaN,...,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.00000,NaN,1.00000
781,31275,OR4F29,ENSG00000235249,1,367660,1.0,1.0,1.0,1.0,NaN,...,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.00000,NaN,1.00000
782,31275,OR4F29,ENSG00000235249,1,367661,1.0,1.0,1.0,1.0,NaN,...,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.00000,NaN,1.00000
783,31275,OR4F29,ENSG00000235249,1,367662,1.0,1.0,1.0,1.0,NaN,...,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.00000,NaN,1.00000
784,31275,OR4F29,ENSG00000235249,1,367663,1.0,1.0,1.0,1.0,NaN,...,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.00000,NaN,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38837009,1809,CDY1,ENSG00000172288,Y,27770670,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26316,NaN,0.13158
38837010,1809,CDY1,ENSG00000172288,Y,27770671,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26316,NaN,0.13158
38837011,1809,CDY1,ENSG00000172288,Y,27770672,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26316,NaN,0.13158
38837012,1809,CDY1,ENSG00000172288,Y,27770673,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.26316,NaN,0.13158


In [2]:
refseq = pd.read_csv('/home/weber/PycharmProjects/ExoCarto/data/2_processed/DF_transformed.csv.gz', compression='gzip', sep='\t').dropna(subset=['HGNC']).sort_values(by=['HGNC', 'Chrom', 'Start'], ascending=True)
refseq['HGNC'] = refseq['HGNC'].astype(int)
refseq

/home/weber/.conda/envs/ExoCarto/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Gene,Chrom,Start,End,ranges,Length,GeneID,HGNC,CDS_representation,Ratio,Sharing_status,Sources,Count_CDS_alternative,Count_CDS_constitutive,mRNA_IDS,mRNA_nb
110056,A1BG,19,58858388,58858395,58858388-58858395,7,1,5,Unique,1.0/1,False,"[(58858388, 58858395, '1.0/1')]",0,8,['NM_130786.4'],1
110057,A1BG,19,58858719,58859006,58858719-58859006,287,1,5,Unique,1.0/1,False,"[(58858719, 58859006, '1.0/1')]",0,8,['NM_130786.4'],1
110058,A1BG,19,58861736,58862017,58861736-58862017,281,1,5,Unique,1.0/1,False,"[(58861736, 58862017, '1.0/1')]",0,8,['NM_130786.4'],1
110059,A1BG,19,58862757,58863053,58862757-58863053,296,1,5,Unique,1.0/1,False,"[(58862757, 58863053, '1.0/1')]",0,8,['NM_130786.4'],1
110060,A1BG,19,58863649,58863921,58863649-58863921,272,1,5,Unique,1.0/1,False,"[(58863649, 58863921, '1.0/1')]",0,8,['NM_130786.4'],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79640,DERPC,16,69152757,69154010,69152757-69154010,1253,113455421,54084,Variable_region,4.0/7,True,"[(69152757, 69154331, '4.0/7')]",2,1,"['NM_001366605.1', 'NM_001366603.1', 'NM_00104...",7
79641,DERPC,16,69154010,69154331,69154010-69154331,321,113455421,54084,Variable_region,7.0/7,True,"[(69152757, 69154331, '4.0/7'), (69154010, 691...",2,1,"['NM_001366605.1', 'NM_001366603.1', 'NM_00104...",7
79642,DERPC,16,69154010,69154331,69154010-69154331,321,113455421,54084,Variable_region,7.0/7,True,"[(69152757, 69154331, '4.0/7'), (69154010, 691...",2,1,"['NM_001366605.1', 'NM_001366603.1', 'NM_00104...",7
9453,LINC02801,1,87709298,87709320,87709298-87709320,22,101927844,54325,Unique,1.0/1,False,"[(87709298, 87709320, '1.0/1')]",0,2,['NM_001290053.1'],1


In [251]:
refseq.loc[refseq['Gene'] == 'DMD']

,Gene,Chrom,Start,End,ranges,Length,GeneID,HGNC,CDS_representation,Ratio,Sharing_status,Sources,Count_CDS_alternative,Count_CDS_constitutive,mRNA_IDS,mRNA_nb,Ratio_num
231896,DMD,X,31139950,31140036,31139950-31140036,86,1756,2928,Variable_region,5.0/17,True,"[(31139950, 31140047, '5.0/17')]",81,8,"['NM_000109.4', 'NM_004011.4', 'NM_004012.4', ...",17,0.294118
231897,DMD,X,31140036,31140047,31140036-31140047,11,1756,2928,Variable_region,16.0/17,True,"[(31139950, 31140047, '5.0/17'), (31140036, 31...",81,8,"['NM_000109.4', 'NM_004011.4', 'NM_004012.4', ...",17,0.941176
231898,DMD,X,31144759,31144790,31144759-31144790,31,1756,2928,Unique,11.0/17,False,"[(31144759, 31144790, '11.0/17')]",81,8,"['NM_000109.4', 'NM_004011.4', 'NM_004012.4', ...",17,0.647059
231899,DMD,X,31152219,31152311,31152219-31152311,92,1756,2928,Unique,16.0/17,False,"[(31152219, 31152311, '16.0/17')]",81,8,"['NM_000109.4', 'NM_004011.4', 'NM_004012.4', ...",17,0.941176
231900,DMD,X,31164408,31164531,31164408-31164531,123,1756,2928,Unique,16.0/17,False,"[(31164408, 31164531, '16.0/17')]",81,8,"['NM_000109.4', 'NM_004011.4', 'NM_004012.4', ...",17,0.941176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231980,DMD,X,32867845,32867937,32867845-32867937,92,1756,2928,Unique,3.0/17,False,"[(32867845, 32867937, '3.0/17')]",81,8,"['NM_000109.4', 'NM_004011.4', 'NM_004012.4', ...",17,0.176471
231981,DMD,X,33038256,33038317,33038256-33038317,61,1756,2928,Unique,3.0/17,False,"[(33038256, 33038317, '3.0/17')]",81,8,"['NM_000109.4', 'NM_004011.4', 'NM_004012.4', ...",17,0.176471
231982,DMD,X,33146264,33146282,33146264-33146282,18,1756,2928,Unique,1.0/17,False,"[(33146264, 33146282, '1.0/17')]",81,8,"['NM_000109.4', 'NM_004011.4', 'NM_004012.4', ...",17,0.058824
231983,DMD,X,33229399,33229429,33229399-33229429,30,1756,2928,Unique,1.0/17,False,"[(33229399, 33229429, '1.0/17')]",81,8,"['NM_000109.4', 'NM_004011.4', 'NM_004012.4', ...",17,0.058824


In [3]:
multi_mrna_genes = refseq.loc[refseq['mRNA_nb'] > 1, 'HGNC'].unique()
multi_mrna_genes

array([    7,    18,    19, ..., 53651, 53757, 54084])

In [105]:



def map_refseq_pext(gene, l):
    tmp_l = list()
    refseq_gene = refseq.loc[refseq['HGNC'] == gene].reset_index(drop=True)
    pext_gene = pext_modified.loc[pext_modified['HGNC ID'] == gene].reset_index(drop=True)
    for r in refseq_gene[['Start', 'End']].values:
        start = r[0]
        end = r[1]
        pext_exon_values = pext_gene.loc[pext_gene['pos'].between(start, end)].groupby(['HGNC ID', 'symbol', 'ensg']).median().drop(['pos'], axis=1)
        pext_exon_values['Exon'] = str(start) + '-' + str(end)
        tmp_l.append(pext_exon_values)
    if tmp_l:
        concat_df_gene = pd.concat(tmp_l).reset_index()
        concat_df_gene = concat_df_gene[list(concat_df_gene.columns[:3]) + ['Exon'] + sorted(list(concat_df_gene.columns[3:-2])) + ['mean_proportion']]
        l.append(concat_df_gene)
    
    
m = multiprocessing.Manager()
l = m.list()
parmap.starmap(map_refseq_pext, list(zip(sorted(refseq.HGNC.unique()))), l, pm_pbar=True, pm_processes=20)
    
output_df = pd.concat(list(l))
output_df.to_parquet('/home/weber/PycharmProjects/ExoCarto/data/clean/2_expression/RefSeq_PEXT.parquet', index=False)
output_df



    # transposed = test.T
    # percentile_df = list()
    # for col in transposed:
    #     percentile_df.append(transposed[col].rank(pct=True))
    # pd.concat(percentile_df, axis=1).sort_index().T




19200it [05:34, 57.41it/s]                            


In [4]:
phylocsf_file = "/gstock/biolo_datasets/variation/gnomAD/latest/gtex/phylocsf_data.tsv.bgz"
phylocsf_df = pd.read_csv(phylocsf_file, compression='gzip', sep='\t')
phylocsf_df = phylocsf_df[['gene_id', 'chromosome_name', 'start_coordinate', 'end_coordinate', 'tag', 'max_score']]
phylocsf_df = phylocsf_df.loc[phylocsf_df['tag'] == 'CCDS']
phylocsf_df['chromosome_name'] = phylocsf_df['chromosome_name'].str.replace('chr', '')
phylocsf_df = phylocsf_df.sort_values(by=['chromosome_name', 'start_coordinate'], ascending=[True, True]).drop_duplicates()
phylocsf_df['range'] = phylocsf_df['start_coordinate'].astype(str) + '-' + phylocsf_df['end_coordinate'].astype(str)
phylocsf_df['gene_id'] = phylocsf_df['gene_id'].str.split('.').apply(lambda r: r[0])
phylocsf_df = pd.merge(phylocsf_df, biomart_ensembl, right_on='Gene stable ID', left_on='gene_id', how='left')
phylocsf_df = phylocsf_df[list(phylocsf_df.columns)[:5] + ['HGNC ID', 'HGNC symbol', 'range', 'max_score']]
phylocsf_df.rename({'range': 'Range'}, axis=1, inplace=True)
# phylocsf_df['HGNC ID'] = phylocsf_df['HGNC ID'].astype(int)
phylocsf_df

,gene_id,chromosome_name,start_coordinate,end_coordinate,tag,HGNC ID,HGNC symbol,Range,max_score
0,ENSG00000187634,1,861321,861392,CCDS,28706,SAMD11,861321-861392,334.9858
1,ENSG00000187634,1,865534,865715,CCDS,28706,SAMD11,865534-865715,576.4071
2,ENSG00000187634,1,866418,866468,CCDS,28706,SAMD11,866418-866468,146.7116
3,ENSG00000187634,1,871151,871275,CCDS,28706,SAMD11,871151-871275,473.3769
4,ENSG00000187634,1,874419,874508,CCDS,28706,SAMD11,874419-874508,112.1655
...,...,...,...,...,...,...,...,...,...
167795,ENSG00000205916,Y,27042651,27042685,CCDS,15966,DAZ4,27042651-27042685,55.4482
167796,ENSG00000185894,Y,27184955,27185060,CCDS,18225,BPY2C,27184955-27185060,5.1943
167797,ENSG00000185894,Y,27187915,27188032,CCDS,18225,BPY2C,27187915-27188032,-14.0672
167798,ENSG00000185894,Y,27190092,27190169,CCDS,18225,BPY2C,27190092-27190169,11.2080


In [7]:
hi_clingen_file = "/gstock/biolo_datasets/variation/benchmark/Databases/ClinGen/ClinGen_haploinsufficiency_gene_GRCh37.bed"
hi_clingen = pd.read_csv(hi_clingen_file, sep='\t', skiprows=1, names=['chrom', 'start', 'end', 'symbol', 'hi_level'])
hi_clingen

,chrom,start,end,symbol,hi_level
0,chr1,100652477,100715409,DBT,30
1,chr1,103342022,103574052,COL11A1,1
2,chr1,10535002,10690815,PEX14,30
3,chr1,10696660,10856733,CASZ1,1
4,chr1,109419602,109476957,GPSM2,30
...,...,...,...,...,...
1397,chrY,25130409,25151612,BPY2,0
1398,chrY,25275501,25345239,DAZ1,0
1399,chrY,26356113,26360978,GOLGA2P2Y,0
1400,chrY,2654895,2655782,SRY,3


In [ ]:
phylocsf_df.loc[phylocsf_df['HGNC symbol'] == 'CACNB2']

In [9]:
vcf_df = pd.read_parquet('/home/weber/PycharmProjects/ExoCarto/data/2_processed/RefSeq_X_VCF_NO_CCRS_pathogenic_only.parquet')
biomart_ensembl['HGNC ID'] = biomart_ensembl['HGNC ID'].astype(int)
vcf_df = pd.merge(vcf_df, biomart_ensembl, right_on='HGNC ID', left_on='RefSeq_HGNC', how='left')
vcf_df

,CCRS_Gene,RefSeq_CDS_representation,RefSeq_Chrom,RefSeq_Count_CDS_alternative,RefSeq_Count_CDS_constitutive,RefSeq_End,RefSeq_GeneID,RefSeq_HGNC,RefSeq_Length,RefSeq_Ratio,...,RefSeq_mRNA_IDS,RefSeq_mRNA_nb,RefSeq_ranges,MC,STATUS,Source,VAR_ID,Gene stable ID,HGNC ID,HGNC symbol
0,A2M,Unique,12,5,33,9266139,2,7,183,0.5,...,"['NM_001347423.2', 'NM_001347425.1', 'NM_00001...",4,9265956-9266139,None,None,None,None,ENSG00000175899,7,A2M
1,A2M,Unique,12,5,33,9243078,2,7,126,1.0,...,"['NM_001347423.2', 'NM_001347425.1', 'NM_00001...",4,9242952-9243078,None,None,None,None,ENSG00000175899,7,A2M
2,A2M,Unique,12,5,33,9244025,2,7,228,1.0,...,"['NM_001347423.2', 'NM_001347425.1', 'NM_00001...",4,9243797-9244025,None,None,None,None,ENSG00000175899,7,A2M
3,A2M,Unique,12,5,33,9246175,2,7,114,1.0,...,"['NM_001347423.2', 'NM_001347425.1', 'NM_00001...",4,9246061-9246175,None,None,None,None,ENSG00000175899,7,A2M
4,A2M,Unique,12,5,33,9247680,2,7,111,1.0,...,"['NM_001347423.2', 'NM_001347425.1', 'NM_00001...",4,9247569-9247680,None,None,None,None,ENSG00000175899,7,A2M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137431,ZPBP,Unique,7,1,8,50070906,11055,15662,218,1.0,...,"['NM_001159878.2', 'NM_007009.3']",2,50070688-50070906,missense,Pathogenic,HGMD,50070744_T_A,ENSG00000042813,15662,ZPBP
137432,ZPBP,Unique,7,1,8,50121495,11055,15662,125,1.0,...,"['NM_001159878.2', 'NM_007009.3']",2,50121370-50121495,None,None,None,None,ENSG00000042813,15662,ZPBP
137433,ZPBP,Variable_region,7,1,8,50097737,11055,15662,3,0.5,...,"['NM_001159878.2', 'NM_007009.3']",2,50097734-50097737,None,None,None,None,ENSG00000042813,15662,ZPBP
137434,ZPBP,Unique,7,1,8,50070906,11055,15662,218,1.0,...,"['NM_001159878.2', 'NM_007009.3']",2,50070688-50070906,missense,Pathogenic,HGMD,50070844_T_C,ENSG00000042813,15662,ZPBP


In [10]:
vcf_df = pd.read_csv('/home/weber/PycharmProjects/ExoCarto/' + 'data/2_processed/DF_transformed.csv.gz', compression='gzip', sep='\t')
vcf_df = vcf_df.rename(
    {
        'Gene' : 'CCRS_Gene',
        'Count_CDS_alternative' : 'RefSeq_Count_CDS_alternative',
        'Count_CDS_constitutive' : 'RefSeq_Count_CDS_constitutive',
        'End' : 'RefSeq_End',
        'GeneID' : 'RefSeq_GeneID',
        'HGNC' : 'RefSeq_HGNC',
        'Length' : 'RefSeq_Length',
        'Ratio' : 'RefSeq_Ratio',
        'Sharing_status' : 'RefSeq_Sharing_status',
        'Sources' : 'RefSeq_Sources',
        'Start' : 'RefSeq_Start',
        'mRNA_IDS' : 'RefSeq_mRNA_IDS',
        'mRNA_nb' : 'RefSeq_mRNA_nb',
        'ranges' : 'RefSeq_ranges',
    },
    axis=1,
)
vcf_df = vcf_df.loc[(vcf_df['RefSeq_mRNA_nb'] > 1) & (vcf_df['RefSeq_HGNC'].isna() == False)].reset_index(drop=True)
vcf_df

,CCRS_Gene,Chrom,RefSeq_Start,RefSeq_End,RefSeq_ranges,RefSeq_Length,RefSeq_GeneID,RefSeq_HGNC,CDS_representation,RefSeq_Ratio,RefSeq_Sharing_status,RefSeq_Sources,RefSeq_Count_CDS_alternative,RefSeq_Count_CDS_constitutive,RefSeq_mRNA_IDS,RefSeq_mRNA_nb
0,PLEKHN1,1,901912,901994,901912-901994,82,84069,25284.0,Unique,3.0/3,False,"[(901912, 901994, '3.0/3')]",2,15,"['NM_001367552.1', 'NM_001160184.2', 'NM_03212...",3
1,PLEKHN1,1,902084,902183,902084-902183,99,84069,25284.0,Unique,3.0/3,False,"[(902084, 902183, '3.0/3')]",2,15,"['NM_001367552.1', 'NM_001160184.2', 'NM_03212...",3
2,PLEKHN1,1,905657,905803,905657-905803,146,84069,25284.0,Unique,3.0/3,False,"[(905657, 905803, '3.0/3')]",2,15,"['NM_001367552.1', 'NM_001160184.2', 'NM_03212...",3
3,PLEKHN1,1,905901,905981,905901-905981,80,84069,25284.0,Unique,3.0/3,False,"[(905901, 905981, '3.0/3')]",2,15,"['NM_001367552.1', 'NM_001160184.2', 'NM_03212...",3
4,PLEKHN1,1,906066,906138,906066-906138,72,84069,25284.0,Unique,3.0/3,False,"[(906066, 906138, '3.0/3')]",2,15,"['NM_001367552.1', 'NM_001160184.2', 'NM_03212...",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160740,CDY1,Y,27768590,27770181,27768590-27770181,1591,9085,1809.0,Variable_region,2.0/2,True,"[(27768590, 27770181, '1.0/2'), (27768590, 277...",2,1,"['NM_004680.2', 'NM_170723.1']",2
160741,CDY1,Y,27770181,27770212,27770181-27770212,31,9085,1809.0,Variable_region,1.0/2,True,"[(27768590, 27770212, '1.0/2')]",2,1,"['NM_004680.2', 'NM_170723.1']",2
160742,CDY1,Y,27770181,27770212,27770181-27770212,31,9085,1809.0,Variable_region,1.0/2,True,"[(27768590, 27770212, '1.0/2')]",2,1,"['NM_004680.2', 'NM_170723.1']",2
160743,CDY1,Y,27770602,27770674,27770602-27770674,72,9085,1809.0,Unique,1.0/2,False,"[(27770602, 27770674, '1.0/2')]",2,1,"['NM_004680.2', 'NM_170723.1']",2


In [12]:
test = pd.merge(vcf_df, hi_clingen, left_on='CCRS_Gene', right_on='symbol')
len(test.CCRS_Gene.unique())

919

In [13]:
genes = vcf_df['RefSeq_HGNC'].unique().tolist()

In [14]:
vcf_df['RefSeq_HGNC'] = vcf_df['RefSeq_HGNC'].astype(int)

pext_modified = pext_modified.dropna(subset=['HGNC ID'])
pext_modified['HGNC ID'] = pext_modified['HGNC ID'].astype(int)

phylocsf_df = phylocsf_df.dropna(subset=['HGNC ID'])
phylocsf_df['HGNC ID'] = phylocsf_df['HGNC ID'].astype(int)

In [23]:
phylocsf_df

,gene_id,chromosome_name,start_coordinate,end_coordinate,tag,HGNC ID,HGNC symbol,Range,max_score
0,ENSG00000187634,1,861321,861392,CCDS,28706,SAMD11,861321-861392,334.9858
1,ENSG00000187634,1,865534,865715,CCDS,28706,SAMD11,865534-865715,576.4071
2,ENSG00000187634,1,866418,866468,CCDS,28706,SAMD11,866418-866468,146.7116
3,ENSG00000187634,1,871151,871275,CCDS,28706,SAMD11,871151-871275,473.3769
4,ENSG00000187634,1,874419,874508,CCDS,28706,SAMD11,874419-874508,112.1655
...,...,...,...,...,...,...,...,...,...
167795,ENSG00000205916,Y,27042651,27042685,CCDS,15966,DAZ4,27042651-27042685,55.4482
167796,ENSG00000185894,Y,27184955,27185060,CCDS,18225,BPY2C,27184955-27185060,5.1943
167797,ENSG00000185894,Y,27187915,27188032,CCDS,18225,BPY2C,27187915-27188032,-14.0672
167798,ENSG00000185894,Y,27190092,27190169,CCDS,18225,BPY2C,27190092-27190169,11.2080


In [ ]:
ptex.loc[ptex['symbol'] == 'A2M', ['Range', 'PTEX_mean_value']].sort_values('Range')

In [ ]:
output_df[['RefSeq_HGNC', 'RefSeq_ranges', 'phyloCSF_max_score', 'PTEX_mean_value']]

In [ ]:
pd.set_option('display.max_rows', 100)
tmp_ptex

In [ ]:
phylocsf_df.loc[phylocsf_df['HGNC symbol'] == 'TP53']

In [15]:
vcf_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160745 entries, 0 to 160744
Data columns (total 16 columns):
CCRS_Gene                        160745 non-null object
Chrom                            160745 non-null object
RefSeq_Start                     160745 non-null int64
RefSeq_End                       160745 non-null int64
RefSeq_ranges                    160745 non-null object
RefSeq_Length                    160745 non-null int64
RefSeq_GeneID                    160745 non-null int64
RefSeq_HGNC                      160745 non-null int64
CDS_representation               160745 non-null object
RefSeq_Ratio                     160745 non-null object
RefSeq_Sharing_status            160745 non-null bool
RefSeq_Sources                   160745 non-null object
RefSeq_Count_CDS_alternative     160745 non-null int64
RefSeq_Count_CDS_constitutive    160745 non-null int64
RefSeq_mRNA_IDS                  160745 non-null object
RefSeq_mRNA_nb                   160745 non-null int64
dty

In [17]:
pext_modified['pos'] = pext_modified['pos'].astype(int)
pext_modified.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36705587 entries, 780 to 38837013
Data columns (total 58 columns):
HGNC ID                                 int64
symbol                                  object
ensg                                    object
chrom                                   object
pos                                     int64
Prostate                                float64
Spleen                                  float64
Brain_FrontalCortex_BA9_                float64
SmallIntestine_TerminalIleum            float64
MinorSalivaryGland                      float64
Artery_Coronary                         float64
Skin_SunExposed_Lowerleg_               float64
Cells_EBV_transformedlymphocytes        float64
Brain_Hippocampus                       float64
Esophagus_Muscularis                    float64
Brain_Nucleusaccumbens_basalganglia_    float64
Artery_Tibial                           float64
Brain_Hypothalamus                      float64
Adipose_Visceral_Omentum_ 

In [ ]:
phylocsf_df.loc[(phylocsf_df['HGNC symbol'] == 'TP53') & (phylocsf_df['start_coordinate'] <= 7579908) & (phylocsf_df['end_coordinate'] >= 7579909)]

In [18]:
def mapping(gene, l):
    tmp_ptex = pext_modified.loc[pext_modified['HGNC ID'] == int(gene)].reset_index(drop=True)
    tmp_phylocsf_df = phylocsf_df.loc[phylocsf_df['HGNC ID'] == int(gene)].reset_index(drop=True)
    tmp_vcf = vcf_df.loc[vcf_df['RefSeq_HGNC'] == int(gene)].sort_values(by='RefSeq_ranges').reset_index(drop=True)

    l_ptex = list()
    l_phylocsf = list()

    for r in tmp_vcf['RefSeq_ranges'].values:
        ptex_values = tmp_ptex.loc[(tmp_ptex['pos'] >= int(r.split('-')[0])) & (tmp_ptex['pos'] <= int(r.split('-')[1])), list(tmp_ptex.columns)[5:]].mean().to_dict()
        l_ptex.append(ptex_values)

        phylocsf_score = tmp_phylocsf_df.loc[(tmp_phylocsf_df['start_coordinate']-1 <= int(r.split('-')[0])) & (int(r.split('-')[1]) <= tmp_phylocsf_df['end_coordinate']+1), 'max_score'].values
        if len(phylocsf_score) > 0:
            l_phylocsf.append({'phyloCSF_max_score' : np.mean(phylocsf_score)})
        else:
            l_phylocsf.append({'phyloCSF_max_score' : np.nan})

    l.append(pd.concat([tmp_vcf, pd.DataFrame(l_ptex), pd.DataFrame(l_phylocsf)], axis=1))

m = multiprocessing.Manager()
mp_l = m.list()
# parmap.starmap(mapping, list(zip([11998])), mp_l, pm_pbar=True)
parmap.starmap(mapping, list(zip(genes)), mp_l, pm_pbar=True)
output_df = pd.concat(list(mp_l))
output_df['Chrom'] = output_df['Chrom'].astype(str)
output_df = output_df.loc[(~output_df.Chrom.str.contains('CHR')) & output_df['mean_proportion'].isna() == False]
output_df



11008it [02:09, 84.78it/s]


,CCRS_Gene,Chrom,RefSeq_Start,RefSeq_End,RefSeq_ranges,RefSeq_Length,RefSeq_GeneID,RefSeq_HGNC,CDS_representation,RefSeq_Ratio,...,Brain_Putamen_basalganglia_,Brain_Caudate_basalganglia_,Colon_Sigmoid,Cervix_Endocervix,Ovary,Esophagus_GastroesophagealJunction,Testis,Brain_Amygdala,mean_proportion,phyloCSF_max_score
0,PIGC,1,172410869,172411762,172410869-172411762,893,5279,8960,Unique,2.0/2,...,0.86556,0.8819,0.90581,0.90592,0.89855,0.92001,0.92957,0.83829,0.90539,NaN
0,BORCS7,10,104614044,104614184,104614044-104614184,140,119032,23516,Unique,2.0/2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,628.8381
1,BORCS7,10,104620089,104620151,104620089-104620151,62,119032,23516,Unique,2.0/2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,289.6790
2,BORCS7,10,104620255,104620298,104620255-104620298,43,119032,23516,Unique,2.0/2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,220.8366
3,BORCS7,10,104621917,104621937,104621917-104621937,20,119032,23516,Unique,2.0/2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.6227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,AGPAT1,CHR_HSCHR6_MHC_QBL_CTG1,32144287,32144289,32144287-32144289,2,10554,324,Unique,1.0/5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,AGPAT1,CHR_HSCHR6_MHC_APD_CTG1,32144287,32144289,32144287-32144289,2,10554,324,Unique,1.0/5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,AGPAT1,CHR_HSCHR6_MHC_MANN_CTG1,32144287,32144289,32144287-32144289,2,10554,324,Unique,1.0/5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,AGPAT1,6,32144287,32144289,32144287-32144289,2,10554,324,Unique,1.0/5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
output_df.loc[output_df['CCRS_Gene'] == 'TP53'].sort_values(by='RefSeq_ranges')[['RefSeq_ranges', 'mean_proportion', 'phyloCSF_max_score']].drop_duplicates().values

In [36]:
output_df['Chrom'] = output_df['Chrom'].astype(str)
output_df = output_df[~output_df.Chrom.str.contains('CHR')]
output_df = output_df.loc[output_df['mean_proportion'].isna() == False]
output_df

,CCRS_Gene,Chrom,RefSeq_Start,RefSeq_End,RefSeq_ranges,RefSeq_Length,RefSeq_GeneID,RefSeq_HGNC,CDS_representation,RefSeq_Ratio,...,Brain_Putamen_basalganglia_,Brain_Caudate_basalganglia_,Colon_Sigmoid,Cervix_Endocervix,Ovary,Esophagus_GastroesophagealJunction,Testis,Brain_Amygdala,mean_proportion,phyloCSF_max_score
0,PIGC,1,172410869,172411762,172410869-172411762,893,5279,8960,Unique,2.0/2,...,0.865560,0.881900,0.905810,0.905920,0.898550,0.920010,0.929570,0.838290,0.905390,NaN
0,ZNF669,1,247263676,247264621,247263676-247264621,945,79862,25736,Unique,2.0/2,...,1.000000,1.000000,1.000000,0.908027,0.964345,1.000000,1.000000,1.000000,0.996064,NaN
1,ZNF669,1,247265033,247265093,247265033-247265093,60,79862,25736,Unique,2.0/2,...,1.000000,1.000000,1.000000,0.898830,0.960780,1.000000,1.000000,1.000000,0.995670,-8.1007
2,ZNF669,1,247265289,247265415,247265289-247265415,126,79862,25736,Unique,2.0/2,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,14.8000
3,ZNF669,1,247267241,247267499,247267241-247267499,258,79862,25736,Variable_region,1.0/2,...,0.003861,0.003861,0.003861,0.104640,0.077106,0.003861,0.065503,0.003861,0.039514,143.2459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,AGPAT1,6,32137942,32138037,32137942-32138037,95,10554,324,Unique,5.0/5,...,0.980250,0.984710,0.982810,0.988680,0.983010,0.978470,0.982300,0.983850,0.984970,391.7055
27,AGPAT1,6,32138202,32138377,32138202-32138377,175,10554,324,Unique,5.0/5,...,0.980250,0.984710,0.982760,0.988603,0.982960,0.978418,0.982263,0.983829,0.984935,754.5904
28,AGPAT1,6,32138714,32138847,32138714-32138847,133,10554,324,Unique,5.0/5,...,0.980250,0.984710,0.978390,0.981860,0.978610,0.973890,0.979080,0.982010,0.981910,534.3331
36,AGPAT1,6,32139074,32139273,32139074-32139273,199,10554,324,Variable_region,5.0/5,...,0.980250,0.984710,0.978390,0.981860,0.978610,0.973890,0.979080,0.982010,0.981910,475.7562


In [37]:
output_df.to_parquet('/home/weber/PycharmProjects/ExoCarto/data/2_processed/refseq_multi_iso_pext_phylocsf.parquet')


In [ ]:
import pandas as pd
tp53_df = pd.read_csv('/home/weber/PycharmProjects/ExoCarto-Augusta/RefSeq_X_VCF_TP53-BRCA1.csv.gz', sep='\t', compression='gzip')
tp53_df = tp53_df.loc[tp53_df['RefSeq_HGNC'] == 11998].sort_values(by='CCRS_ranges')
tp53_df.CCRS_ranges.unique()